# Exploratory Data Analysis One
### In this notebook, I will be bringing in data, arranging it so that it can be used properly, and then exporting it to a .json file so I can use it on the cloud. 

#### Importing the necessary libraries

In [118]:
#DATA WRANGLING
import pandas as pd # Dataframes
from pandas.io.json import json_normalize # JSON wrangler
import statsapi # Python wrapper MLB data API

In [119]:
#DATA STORAGE
#from sqlalchemy import create_engine # SQL helper
import psycopg2 as psql #PostgreSQL DBs

In [120]:
#DATA MANIPULATION AND MODELLING
import numpy as np
np.random.seed(0)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn_pandas import DataFrameMapper, FunctionTransformer, gen_features, pipeline
from sklearn_pandas.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import seaborn as sns
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelBinarizer
import xgboost as xgb
import os

In [121]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [122]:
#statsapi.schedule

In [123]:
team_list = ['108','136']

## Data Extraction
Use the StatsApi **insert hyperlink** to collect infomation about all games played between start and ending of the season(use actual) dates. We need to collect .json file per mlb team, so I used a for loop will load and write out the schedule data as a .json file. 


Importing team codes from TSV file in Public Data direcrtory. Then edit the team names so that they are lowercase and seperated by an underscore. 

In [124]:
teams = pd.read_csv('public_data/team_codes.tsv', sep='\t')

teams['full_name'] = teams['full_name'].str.lower().str.replace(' ', '_').str.replace('-','_')

In [125]:
teams.head()

,code,short_name,full_name
0,108,LAA,angels
1,109,ARI,d_backs
2,110,BAL,orioles
3,111,BOS,red_sox
4,112,CHC,cubs


In [126]:
for code, team_name in zip(teams['code'],teams['full_name']):
    print(f'The {team_name} has code {code}')

The angels has code 108
The d_backs has code 109
The orioles has code 110
The red_sox has code 111
The cubs has code 112
The reds has code 113
The indians has code 114
The rockies has code 115
The tigers has code 116
The astros has code 117
The royals has code 118
The dodgers has code 119
The nationals has code 120
The mets has code 121
The athletics has code 133
The pirates has code 134
The padres has code 135
The mariners has code 136
The giants has code 137
The cardinals has code 138
The rays has code 139
The rangers has code 140
The blue_jays has code 141
The twins has code 142
The phillies has code 143
The braves has code 144
The white_sox has code 145
The marlins has code 146
The yankees has code 147
The brewers has code 158


In [127]:
schedule = statsapi.schedule(start_date="03/28/2018", end_date="04/01/2018")


In [128]:
full = json_normalize(schedule)
gamepks= full['game_id']

In [129]:
full.head(2)

,away_id,away_name,away_pitcher_note,away_probable_pitcher,away_score,current_inning,doubleheader,game_date,game_datetime,game_id,game_num,game_type,home_id,home_name,home_pitcher_note,home_probable_pitcher,home_score,inning_state,losing_pitcher,losing_team,save_pitcher,status,summary,winning_pitcher,winning_team
0,112,Chicago Cubs,Lester will make his 7th Opening Day start and...,"Lester, Jon",8,9,N,2018-03-29,2018-03-29T16:40:00Z,529407,1,R,146,Miami Marlins,"In 2017, Urena entered Spring Training out of ...","Urena, Jose",4,Bottom,Jose Urena,Miami Marlins,None,Final,2018-03-29 - Chicago Cubs (8) @ Miami Marlins ...,Steve Cishek,Chicago Cubs
1,134,Pittsburgh Pirates,Nova will make his first Opening Day start at ...,"Nova, Ivan",0,,N,2018-03-29,2018-03-29T17:10:00Z,529417,1,R,116,Detroit Tigers,Zimmermann makes his first Opening Day start a...,"Zimmermann, Jordan",0,,NaN,NaN,NaN,Postponed,2018-03-29 - Pittsburgh Pirates @ Detroit Tige...,NaN,NaN


In [130]:
gamepks_2018 = list(gamepks.unique())
len(gamepks_2018)

51

In [131]:
test_pk = gamepks_2018[:6]

In [132]:
pd.set_option('display.max_columns', None)

In [133]:
test_pk

[529407, 529417, 529419, 529414, 529411, 529415]

This for loop is designed to bring in the play by play data from each game, which is written in .json and it will flatten the .json out in levels. I defined columns names from the original data pulled in and I'm going to use the two flattened levels in order to make one cohesive dataframe with all of the columns that I would like. 

In [136]:
list_for_final_df = []


for game in test_pk:
    print(game)
    curr_game = statsapi.get('game_playByPlay',{'gamePk':game})
    curr_plays = curr_game.get('allPlays')
    curr_plays_df = pd.DataFrame(curr_plays)
    curr_plays_norm = json_normalize(curr_plays)
    
    all_plays_cols = ['about.atBatIndex', 'about.halfInning', 'about.Inning', 'count.balls', 'count.strikes', 'matchup.batSide.code', 
                     'matchup.batter.fullName', 'matchup.batter.id', 'matchup.pitchHand.code', 'matchup.pitcher.fullName',
                     'matchup.pitcher.id', 'result.eventType']
    
    play_events_cols = ['count.balls', 'count.strikes', 'details.ballColor', 'details.call.code', 'details.call.description',
                        'detail.call.code', 'details.description', 'details.code', 'details.type.code', 'index', 'pitchData.nastyFactor',
                       'pitchData.zone', 'pitchNumber', 'type']
    i = 1
    for index, row in curr_plays_norm.iterrows():
            play_events = json_normalize(row['playEvents'])
            
            for play_events_idx, play_events_row in play_events.iterrows():
                
                game_dict = {}
                game_dict['gamepk'] = game
                game_dict['pitch_id']  = str(game) + '_' + str(row['about.atBatIndex']) + '_' + str(i)
                game_dict['prior_pitch'] = str(game) + '_' + (str(row['about.atBatIndex']) + '_' + str(i - 1))
                
                for col_all_plays in all_plays_cols:
                    if col_all_plays in curr_plays_norm.columns:
                        game_dict[col_all_plays] = row[col_all_plays]
                    else:
                        game_dict[col_all_plays] = np.nan
                for col_play_events in play_events_cols:
                    if col_play_events in play_events.columns:
                        game_dict[col_play_events] = play_events_row[col_play_events]
                    else: 
                        game_dict[col_play_events] = np.nan
                
                list_for_final_df.append(game_dict)
                i += 1
                                                              
                                                              
                
                                                            
            
            

        
    


529407
529417
529419
529414
529411
529415


In [140]:
each_pitch = pd.DataFrame(list_for_final_df)
each_pitch.head(10)

,about.Inning,about.atBatIndex,about.halfInning,count.balls,count.strikes,detail.call.code,details.ballColor,details.call.code,details.call.description,details.code,details.description,details.type.code,gamepk,index,matchup.batSide.code,matchup.batter.fullName,matchup.batter.id,matchup.pitchHand.code,matchup.pitcher.fullName,matchup.pitcher.id,pitchData.nastyFactor,pitchData.zone,pitchNumber,pitch_id,prior_pitch,result.eventType,type
0,NaN,0,top,0.0,0.0,NaN,"rgba(26, 86, 190, 1.0)",X,Hit Into Play - Out(s),E,"In play, run(s)",FT,529407,0,L,Ian Happ,664023,R,Jose Urena,570632,32.89,6.0,1.0,529407_0_1,529407_0_0,home_run,pitch
1,NaN,1,top,1.0,0.0,NaN,"rgba(39, 161, 39, 1.0)",B,Ball - Called,B,Ball,FT,529407,0,R,Kris Bryant,592178,R,Jose Urena,570632,24.17,13.0,1.0,529407_1_2,529407_1_1,walk,pitch
2,NaN,1,top,2.0,0.0,NaN,"rgba(39, 161, 39, 1.0)",B,Ball - Called,B,Ball,FT,529407,1,R,Kris Bryant,592178,R,Jose Urena,570632,29.02,13.0,2.0,529407_1_3,529407_1_2,walk,pitch
3,NaN,1,top,2.0,1.0,NaN,"rgba(170, 21, 11, 1.0)",S,Strike - Swinging,S,Swinging Strike,FT,529407,2,R,Kris Bryant,592178,R,Jose Urena,570632,41.63,13.0,3.0,529407_1_4,529407_1_3,walk,pitch
4,NaN,1,top,3.0,1.0,NaN,"rgba(39, 161, 39, 1.0)",B,Ball - Called,B,Ball,CH,529407,3,R,Kris Bryant,592178,R,Jose Urena,570632,59.33,13.0,4.0,529407_1_5,529407_1_4,walk,pitch
5,NaN,1,top,3.0,2.0,NaN,"rgba(170, 21, 11, 1.0)",S,Strike - Swinging,F,Foul,FT,529407,4,R,Kris Bryant,592178,R,Jose Urena,570632,39.41,4.0,5.0,529407_1_6,529407_1_5,walk,pitch
6,NaN,1,top,4.0,2.0,NaN,"rgba(39, 161, 39, 1.0)",B,Ball - Called,B,Ball,CH,529407,5,R,Kris Bryant,592178,R,Jose Urena,570632,14.88,11.0,6.0,529407_1_7,529407_1_6,walk,pitch
7,NaN,2,top,0.0,1.0,NaN,"rgba(170, 21, 11, 1.0)",S,Strike - Swinging,F,Foul,FT,529407,0,L,Anthony Rizzo,519203,R,Jose Urena,570632,45.62,9.0,1.0,529407_2_8,529407_2_7,hit_by_pitch,pitch
8,NaN,2,top,0.0,2.0,NaN,"rgba(170, 21, 11, 1.0)",S,Strike - Swinging,F,Foul,FT,529407,1,L,Anthony Rizzo,519203,R,Jose Urena,570632,31.03,6.0,2.0,529407_2_9,529407_2_8,hit_by_pitch,pitch
9,NaN,2,top,1.0,2.0,NaN,"rgba(39, 161, 39, 1.0)",B,Ball - Called,B,Ball,FT,529407,2,L,Anthony Rizzo,519203,R,Jose Urena,570632,53.95,12.0,3.0,529407_2_10,529407_2_9,hit_by_pitch,pitch


In [ ]:
## Next step is to get the previous pitch column locked in
## Then begin working on how to get the players stats as part of the dataframe. 

In [112]:
curr_plays_norm.head(2)

,about.atBatIndex,about.captivatingIndex,about.endTime,about.halfInning,about.hasOut,about.hasReview,about.inning,about.isComplete,about.isScoringPlay,about.startTime,actionIndex,atBatIndex,count.balls,count.outs,count.strikes,matchup.batSide.code,matchup.batSide.description,matchup.batter.fullName,matchup.batter.id,matchup.batter.link,matchup.batterHotColdZoneStats.stats,matchup.batterHotColdZones,matchup.pitchHand.code,matchup.pitchHand.description,matchup.pitcher.fullName,matchup.pitcher.id,matchup.pitcher.link,matchup.pitcherHotColdZoneStats.stats,matchup.pitcherHotColdZones,matchup.splits.batter,matchup.splits.menOnBase,matchup.splits.pitcher,pitchIndex,playEndTime,playEvents,result.awayScore,result.description,result.event,result.eventType,result.homeScore,result.rbi,result.type,runnerIndex,runners
0,0,0,2018-03-29T20:11:04.000Z,top,False,False,1,True,False,2018-03-29T20:08:59.000Z,[],0,2,0,2,L,Left,Brett Gardner,458731,/api/v1/people/458731,NaN,[],L,Left,J.A. Happ,457918,/api/v1/people/457918,NaN,[],vs_LHP,Men_On,vs_LHB,"[0, 1, 2, 3, 4]",2018-03-29T20:11:04.000Z,"[{'details': {'call': {'code': 'S', 'descripti...",0,Brett Gardner reaches on a fielding error by l...,Field Error,field_error,0,0,atBat,[0],"[{'movement': {'start': None, 'end': '1B', 'ou..."
1,1,14,2018-03-29T20:13:10.000Z,top,True,False,1,True,False,2018-03-29T20:11:05.000Z,[],1,0,1,3,R,Right,Aaron Judge,592450,/api/v1/people/592450,NaN,[],L,Left,J.A. Happ,457918,/api/v1/people/457918,NaN,[],vs_LHP,Men_On,vs_RHB,"[0, 1, 2, 3]",2018-03-29T20:13:10.000Z,[{'details': {'description': 'Pickoff Attempt ...,0,Aaron Judge strikes out swinging.,Strikeout,strikeout,0,0,atBat,[0],"[{'movement': {'start': None, 'end': None, 'ou..."


In [113]:
play_events

,count.balls,count.strikes,details.ballColor,details.call.code,details.call.description,details.code,details.description,details.hasReview,details.isBall,details.isInPlay,details.isStrike,details.trailColor,details.type.code,details.type.description,endTime,index,isPitch,pfxId,pitchData.breaks.breakAngle,pitchData.breaks.breakLength,pitchData.breaks.breakY,pitchData.breaks.spinDirection,pitchData.breaks.spinRate,pitchData.coordinates.aX,pitchData.coordinates.aY,pitchData.coordinates.aZ,pitchData.coordinates.pX,pitchData.coordinates.pZ,pitchData.coordinates.pfxX,pitchData.coordinates.pfxZ,pitchData.coordinates.vX0,pitchData.coordinates.vY0,pitchData.coordinates.vZ0,pitchData.coordinates.x,pitchData.coordinates.x0,pitchData.coordinates.y,pitchData.coordinates.y0,pitchData.coordinates.z0,pitchData.endSpeed,pitchData.nastyFactor,pitchData.startSpeed,pitchData.strikeZoneBottom,pitchData.strikeZoneTop,pitchData.typeConfidence,pitchData.zone,pitchNumber,playId,startTime,type
0,0,1,"rgba(170, 21, 11, 1.0)",S,Strike - Swinging,F,Foul,False,False,False,True,"rgba(0, 0, 254, 1.0)",SL,Slider,2018-03-29T22:58:59.000Z,0,True,180329_225837,25.2,7.2,24.0,252,2578.0,-13.19,24.01,-27.55,-0.80,2.61,-7.91,2.77,-1.04,-127.72,-3.41,147.33,0.65,168.35,50.0,6.11,81.3,70.59,87.6,1.45,3.32,2.0,11,1,92456712-af13-4352-ad85-d98e602b0911,2018-03-29T22:58:27.000Z,pitch
1,0,2,"rgba(170, 21, 11, 1.0)",S,Strike - Swinging,S,Swinging Strike,False,False,False,True,"rgba(0, 0, 254, 1.0)",SL,Slider,2018-03-29T22:59:22.000Z,1,True,180329_225904,13.2,8.4,24.0,264,NaN,-7.09,25.23,-30.59,-0.83,1.35,-4.23,0.94,-2.56,-128.29,-5.79,148.48,0.73,202.40,50.0,6.00,81.7,54.34,88.1,1.45,3.32,2.0,13,2,448105f0-f5dd-440b-b041-7e488154974d,2018-03-29T22:58:59.000Z,pitch
2,0,2,"rgba(170, 21, 11, 1.0)",S,Strike - Swinging,F,Foul,False,False,False,True,"rgba(50, 0, 221, 1.0)",SI,Sinker,2018-03-29T23:00:29.000Z,2,True,180329_225926,46.8,4.8,24.0,136,2644.0,17.34,34.70,-14.33,-0.02,4.15,8.04,8.28,-3.99,-145.96,-3.27,117.88,0.32,126.80,50.0,6.15,91.5,18.68,100.2,1.45,3.32,2.0,11,3,9000b967-6218-44db-bf1e-5afad0e3fb44,2018-03-29T22:59:22.000Z,pitch
3,0,3,"rgba(170, 21, 11, 1.0)",S,Strike - Swinging,S,Swinging Strike,False,False,False,True,"rgba(50, 0, 221, 1.0)",SI,Sinker,2018-03-29T23:00:37.000Z,3,True,180329_230034,42.0,3.6,24.0,145,2556.0,13.17,33.46,-13.50,0.30,4.29,6.18,8.76,-2.73,-145.00,-3.12,105.55,0.45,122.86,50.0,6.21,91.1,29.56,99.5,1.68,3.24,2.0,12,4,ad9e4470-8ede-47a3-b1a8-d72316dad446,2018-03-29T23:00:29.000Z,pitch


In [ ]:
# Get one game from API
list_for_new_df = []
#gamepks = [566389]
for game in gamepks:
    #print(game)
    i = 1
    curr_game = statsapi.get('game_playByPlay',{'gamePk':game})

    ### 3. Extract play-by-play data and store into dataframe.

    # Only care about the allPlays key 
    curr_plays = curr_game.get('allPlays')

    # Coerce all plays into a df
    curr_plays_df = json_normalize(curr_plays)

    ###################################
    # Build target table
    ###################################


    # Data from allPlays
    ap_sel_cols = ['about.atBatIndex', 'about.halfInning', 'about.inning', 'result.awayScore', 'result.homeScore'
                   ,'matchup.splits.menOnBase', 'matchup.batSide.code', 'matchup.pitchHand.code', 'count.balls'
              ,'count.strikes', 'count.outs']

    # Data from playEvents
    plev_sel_cols = ['details.type.code', 'details.type.description',  
             'isPitch', 'pitchNumber'
            
           ]

    # Now go through each row. If there is nested list, json_normalize it
    #for index, row in test_df.head(2).iterrows(): #Just using first 2 rows for testing
    for index, row in curr_plays_df.iterrows(): #Just using first 2 rows for testing

        # saw playEvents is a nested list, so json_normalize it
        play_events_df = json_normalize(row['playEvents'])

        #     # look at runners
        #     runners_df = json_normalize(row['runners'])

        # Loop through THIS NESTED dataframe and NOW build the row for the new df    
        for plev_ind, plev_row in play_events_df.iterrows():

            # Instantiate new dict, which will be a single row in target df
            curr_dict = {}
            curr_dict['gamepk'] = game
            curr_dict['pitch_id'] = str(game) + '_' + str(row['about.atBatIndex']) + '_' + str(i)
            curr_dict['prior_pitch'] = str(game) + '_' + str(row['about.atBatIndex']) + '_' + str(i-1)
            

            # Loop through each list, adding their respective values to curr_dict
            
            for col_ap in ap_sel_cols:
                if col_ap in curr_plays_df.columns:
                    curr_dict[col_ap] = row[col_ap]
                else:
                    curr_dict[col_ap] = np.nan
                #print(row['about.atBatIndex'])

            for col_plev in plev_sel_cols:
                if col_plev in play_events_df.columns:
                    curr_dict[col_plev] = plev_row[col_plev]
                else:
                    curr_dict[col_plev] = np.nan

            # collect row dictionary into list
            list_for_new_df.append(curr_dict)
            i += 1


In [64]:
len(list_for_new_df)

17042

In [65]:
pitches_df = pd.DataFrame(list_for_new_df)

In [66]:
pitches_df.head(10)

,about.atBatIndex,count.balls,count.outs,count.strikes,details.call.code,details.call.description,details.isBall,details.isStrike,details.type.code,details.type.description,...,pitchData.coordinates.vZ0,pitchData.coordinates.x,pitchData.coordinates.x0,pitchData.coordinates.y,pitchData.coordinates.y0,pitchData.coordinates.z0,pitchData.endSpeed,pitchData.startSpeed,pitchData.zone,pitchNumber
0,0,0,0,0,X,Hit Into Play - Out(s),False,False,FT,Two-Seam Fastball,...,-6.03,107.75,-1.31,170.38,50.0,5.86,87.1,95.5,6.0,1.0
1,1,4,0,2,B,Ball - Called,True,False,FT,Two-Seam Fastball,...,-7.20,210.19,-1.75,172.62,50.0,6.01,86.3,95.3,13.0,1.0
2,1,4,0,2,B,Ball - Called,True,False,FT,Two-Seam Fastball,...,-7.19,185.57,-1.64,180.77,50.0,5.92,85.5,95.0,13.0,2.0
3,1,4,0,2,S,Strike - Swinging,False,True,FT,Two-Seam Fastball,...,-9.17,169.64,-1.64,203.82,50.0,5.83,85.5,94.5,13.0,3.0
4,1,4,0,2,B,Ball - Called,True,False,CH,Changeup,...,-6.81,152.85,-1.63,195.63,50.0,5.80,80.8,89.5,13.0,4.0
5,1,4,0,2,S,Strike - Swinging,False,True,FT,Two-Seam Fastball,...,-5.27,130.66,-1.67,166.19,50.0,5.86,85.5,94.2,4.0,5.0
6,1,4,0,2,B,Ball - Called,True,False,CH,Changeup,...,-4.38,174.61,-1.59,167.09,50.0,5.94,80.8,89.4,11.0,6.0
7,2,2,0,2,S,Strike - Swinging,False,True,FT,Two-Seam Fastball,...,-6.86,98.41,-1.26,179.11,50.0,5.86,85.4,95.0,9.0,1.0
8,2,2,0,2,S,Strike - Swinging,False,True,FT,Two-Seam Fastball,...,-6.04,107.13,-1.14,168.86,50.0,5.96,86.0,95.7,6.0,2.0
9,2,2,0,2,B,Ball - Called,True,False,FT,Two-Seam Fastball,...,-5.49,86.98,-1.07,162.45,50.0,5.99,86.7,95.5,12.0,3.0
